# pseudo code 

In [ ]:
다른 버스들과 마찬가지로 1112번도 사색에서 도착정보(사색의 광장에 버스가 도착하기 전까지 몇분 남았는지)를 수집할 수 없다. 
그래서 생대역의 도착 정보를 이용할 것이다.
생대역에서 도착정보 없음 계속 뜨다가 처음 정상처리이 뜨는 순간이 사색에 버스가 도착했다는 뜻이다. 그때마다 시간을 기록할 것이다. 
그리고 사색의 광장에 버스가 도착했을 당시에, 운행중인 1112번 버스들의 위치(정류장)도 함께 수집해서 규칙성이 존재하는지 볼 것이다. 
이 코드의 특징:
1112번외에도 다른 버스들도 똑같이 수집 가능하고,
불연속 점을 도출하기 위해 초마다 api 호출을 안해도 되고, 
[사색 도착시간,운행중인 ㅇㅇ버스들의 현 위치]를 수집할 수 있다.

past=없다
1분마다 요청을 한다.
    result=도착정보가 있다 or 없다
    if past!=result:
        if result==도착정보 있다
            처음으로 '있다'가 나왔다. '현시간=사색 도착 시간'을 저장한다. 
    past=result 값 임시 저장

# 실제코드

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

bus_time=[]
p5100 ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
             'routeId':'200000115','stationId':'228000704'}#5100 생과대
p1112 ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
         'routeId':'234000016','stationId':'228000704'}#1112 생과대

def repet(params): 
    url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
    response = requests.get(url, params=params);
    soup=BeautifulSoup(response.text,'xml')
    return soup

def current_buses(): # 사색의 광장에 버스가 도착했을 때, 운행중인 같은 노선의 버스들의 위치(정류장)를 수집  
    url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
    params ={'serviceKey' : 'Jw+++HTzL3V8lfZPcQPb5jB5Rae2NH3BYowjBY+cgMIA1yDcwJNfmtfO+E+O+whPsGCGbJmmpf/X904L4uwGIw==',
              'routeId':'200000115'}#5100번 노선id
    response=requests.get(url,params=params)
    soup=BeautifulSoup(response.text,'xml')
    locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
    return locations


In [3]:
past=4
for x in range(160): # 10분간
    soup=repet(p5100)
    try:
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        bus_time.append([soup.queryTime.text,current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",bus_time)
        time.sleep(60)
    except Exception as e:
        continue
df=pd.DataFrame(bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])
df.to_csv('5100_tue712.csv',index=False)
df

0 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0]]
1 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 

9 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 38

16 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 3

21 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 3

25 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 3

...

155 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 

156 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 

157 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 

158 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 

159 번째 시도:  [['2022-07-12 13:25:29.693', [3, 13, 16, 21, 28, 31, 40, 46], 0], ['2022-07-12 13:26:30.074', [4, 14, 17, 21, 28, 31, 41, 47], 0], ['2022-07-12 13:27:30.408', [5, 14, 17, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:28:31.258', [7, 15, 18, 22, 29, 32, 41, 48], 0], ['2022-07-12 13:29:31.576', [8, 15, 19, 23, 29, 32, 42, 49], 0], ['2022-07-12 13:30:35.082', [1, 8, 15, 19, 23, 29, 32, 42, 49], 1], ['2022-07-12 13:31:35.650', [3, 8, 16, 20, 23, 30, 32, 43, 49], 0], ['2022-07-12 13:32:36.049', [3, 9, 16, 21, 24, 30, 33, 44, 50], 0], ['2022-07-12 13:33:36.677', [3, 9, 16, 21, 24, 31, 34, 45, 51], 0], ['2022-07-12 13:35:56.014', [5, 11, 17, 22, 25, 31, 35, 47, 54], 0], ['2022-07-12 13:36:56.426', [6, 12, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:37:56.604', [7, 13, 18, 23, 26, 32, 36, 48], 0], ['2022-07-12 13:38:56.829', [1, 8, 13, 19, 23, 27, 32, 37, 49], 1], ['2022-07-12 13:39:57.101', [2, 9, 13, 20, 24, 27, 32, 37, 49], 0], ['2022-07-12 13:40:57.326', [3, 9, 14, 21, 24, 27, 32, 

# [단계2]feature engineering

In [235]:
import sklearn
import pandas as pd
from datetime import datetime
import warnings 
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore')

def trim_data(FileName):
    df=pd.read_csv(FileName)
    df['시간'] = df['시간'].apply(pd.to_datetime) # string 값인 시간을 datetime객체로 바꾼다. 
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['second']=df['시간'].dt.second
    df['day'] = df['시간'].dt.weekday #월요일 0 화요일 1

    # unpack list values
    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)

    #7/12 -----------------------------------
    batch=df.index[df.arrived==1]
    temp=0
    for x in batch: # 4 12 27 ...
        for y in range(temp,x): #0-3
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0
    # 7/12 -----------------------------------

    df = df.set_index('시간')
    df.to_csv(f'./사색역데이터/{FileName}')
    return df

In [237]:
import sklearn.preprocessing


df=trim_data('5100_thu.csv')
data=trim_data('5100_fri.csv')

x_train=df.iloc[:,1:]
y_train=df['arrived']
x_test=data.iloc[:,1:]
y_test=data['arrived']

from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=2)
knn_model.fit(x_train, y_train)
knn_prediction = knn_model.predict(x_test)

for x,y in zip(knn_prediction,y_test):
    print(str(x)[:4],y,'차이는 ',x-y)

212. 257 차이는  -44.5
212. 197 차이는  15.5
212. 136 차이는  76.5
151. 75 차이는  76.0
30.0 0 차이는  30.0
391. 425 차이는  -34.0
331. 364 차이는  -33.0
331. 304 차이는  27.0
210. 243 차이는  -32.5
210. 183 차이는  27.5
90.0 123 차이는  -33.0
30.0 63 차이는  -33.0
428. 0 차이는  428.5
827. 903 차이는  -76.0
767. 843 차이는  -76.0
767. 783 차이는  -16.0
616. 722 차이는  -105.5
526. 662 차이는  -136.0
526. 602 차이는  -76.0
496. 541 차이는  -45.0
466. 481 차이는  -15.0
406. 421 차이는  -15.0
406. 361 차이는  45.0
406. 301 차이는  105.0
406. 240 차이는  166.0
406. 180 차이는  226.0
406. 120 차이는  286.0
406. 60 차이는  346.0
406. 0 차이는  406.0
218. 421 차이는  -203.0
893. 361 차이는  532.0
833. 300 차이는  533.0
833. 240 차이는  593.0
833. 180 차이는  653.0
833. 120 차이는  713.0
682. 60 차이는  622.5
682. 0 차이는  682.5
682. 938 차이는  -255.5
682. 877 차이는  -194.5
682. 817 차이는  -134.5
431. 757 차이는  -326.0
341. 697 차이는  -356.0
391. 621 차이는  -230.0
240. 559 차이는  -318.5
150. 497 차이는  -347.0
451. 422 차이는  29.0
451. 361 차이는  90.0
300. 301 차이는  -0.5
210. 241 차이는  -31.0
210. 180 차이는  30.0
90.0 120 차이는